<a href="https://colab.research.google.com/github/ElaineH310/therapy_bot/blob/main/Bot_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing google drive to colab
from google.colab import drive
drive.mount('/content/gdrive')
# to change the current working directory for the notebook environment
%cd '/content/gdrive/MyDrive/Colab Notebooks/SciFair/NLP'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/SciFair/NLP


In [ ]:
!pip install numpy
!pip install nltk
!pip install tensorflow

!pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing
!pip install Pillow==9.5.0

  Cloning https://github.com/MihaMarkic/tflearn.git (to revision fix/is_sequence_missing) to /tmp/pip-req-build-4be8xi69
  Running command git clone --filter=blob:none --quiet https://github.com/MihaMarkic/tflearn.git /tmp/pip-req-build-4be8xi69
  Running command git checkout -b fix/is_sequence_missing --track origin/fix/is_sequence_missing
  Switched to a new branch 'fix/is_sequence_missing'
  Branch 'fix/is_sequence_missing' set up to track remote branch 'fix/is_sequence_missing' from 'origin'.
  Resolved https://github.com/MihaMarkic/tflearn.git to commit 6472b8588e758ff4a33a2764d4ee638bbd0e42f0
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=130659 sha256=db7a645f0b9e224890077415776dcdf588111d9b77cdab0517796f435cc6b431
  Stored in directory: /tmp/pip-ephem-wheel-cache-z3knjuhk/wheels/3f/c2/5d/cf0be63d86a0a58bd81a5c9d71531455fc1a46176b90c2adb6
Successfully built tflearn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4

In [ ]:
import nltk
nltk.download('punkt')
#from nltk.stem.lancaster import LancasterStemmer
#stemmer=LancasterStemmer()
import numpy
# import tflearn #only works in local
import tensorflow
import random
import json
with open('intents_ZH-TW.json') as file:
    data=json.load(file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import jieba
nltk.download('punkt_tab')
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        # 完全使用 jieba 進行斷詞
        wrds = list(jieba.cut(pattern))
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append(intent["tag"])

# 過濾符號，並統一斷詞結果
words = [word for word in words if word not in "?.,!"]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    # 使用 jieba 斷詞處理 docs_x
    wrds = [word for word in jieba.cut(' '.join(doc))]
    for w in words:
        bag.append(1 if w in wrds else 0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
#LancasterStemmer version
"""
words=[]
labels=[]
docs_x=[]
docs_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append (intent["tag"])
words=[stemmer.stem(w.lower()) for w in words if w !="?"]
words=sorted(list(set(words)))
labels=sorted(labels)
training=[]
output=[]
out_empty=[0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1

    training.append(bag)
    output.append(output_row)
"""

In [ ]:
training=numpy.array(training)
output=numpy.array(output)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
import tflearn

K.clear_session()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [ ]:
model.fit(training,output,n_epoch=1500,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 17999  | total loss: 0.00002 | time: 0.045s
| Adam | epoch: 1500 | loss: 0.00002 - acc: 1.0000 -- iter: 88/93
Training Step: 18000  | total loss: 0.00002 | time: 0.049s
| Adam | epoch: 1500 | loss: 0.00002 - acc: 1.0000 -- iter: 93/93
--


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    # 使用 jieba 進行中文分詞
    s_words = list(jieba.cut(s))

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)


In [ ]:
#LancasterStemmer
"""
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)
"""

In [ ]:
!pip install SpeechRecognition
!pip install gTTS

In [ ]:
!pip install ffmpeg-python

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")

  # Check if data contains a comma before splitting
  if ',' in data:
    binary = b64decode(data.split(',')[1])
  else:
    # Handle the case where data doesn't contain a comma
    # For example, you might want to raise an exception or return an error value
    # Here, we're assuming the data is already base64 encoded and doesn't need splitting
    binary = b64decode(data)

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()


In [ ]:
!pip install whisper
!pip install sounddevice wavio
!pip install git+https://github.com/openai/whisper.git
!apt-get install libportaudio2

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-x93zdez_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-x93zdez_
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import random
import time
import numpy as np
import whisper
import sounddevice as sd
import wavio
import os


# Load Whisper model (you can choose 'base', 'small', 'medium', 'large')
whisper_model = whisper.load_model("medium")

# Function to record audio from the microphone
def record_audio(duration=5):
    print(f"Recording for {duration} seconds...")

    # Call get_audio to record audio
    audio, sr = get_audio()

    # Save the recorded audio as a WAV file
    filename = "recorded_audio.wav"
    wavio.write(filename, audio, sr, sampwidth=3)  # Save as WAV file

    return filename


# Function to transcribe the recorded audio using Whisper
def transcribe_audio(filename):
    result = whisper_model.transcribe(filename, language='zh')  # Specify Chinese language
    return result['text']

# def speak(text):
#     tts = gTTS(text=text, lang='zh', slow=False)  # Adjust 'lang' as needed
#     filename = "temp_audio.mp3"
#     tts.save(filename)
#     os.system(f"start {filename}")  # For Windows; use 'xdg-open' for Linux, 'afplay' for macOS

# Main chat function
def chat():
    print("開始與您的機器人對話（輸入 結束 以停止）：")
    #speak("開始與您的機器人對話，輸入結束以停止")
    time.sleep(2)

    while True:
        try:
            choice = input("輸入 1 以進行文字輸入， 2 以進行語音輸入, 結束 以結束對話：")
            if choice == '1':  # Text input
                inp = input("請輸入您的問題：")
            elif choice == '2':  # Speech input
                filename = record_audio(duration=5)
                inp = transcribe_audio(filename)  # Transcribe the recorded audio
                print("You (語音):", inp)
            elif choice=="結束":
                #speak("再見！")
                print("再見！")
                break
            else:
                print("無效的選擇，請輸入 '1' 或 '2'。")
                continue  # Skip to the next iteration
            time.sleep(2)

            # Process the input for the chatbot
            results = model.predict([bag_of_words(inp, words)])
            results_index = np.argmax(results)
            tag = labels[results_index]

            for tg in data["intents"]:
                if tg["tag"] == tag:
                    responses = tg["responses"]

            rd = random.choice(responses)
            #speak(rd)  # Speak the response
            print(rd)
            time.sleep(2)

        except Exception as e:
            #speak("抱歉，發生了錯誤")
            print("發生錯誤:", e)


100%|██████████████████████████████████████| 1.42G/1.42G [00:10<00:00, 140MiB/s]


In [ ]:
chat()

開始與您的機器人對話（輸入 結束 以停止）：
輸入 1 以進行文字輸入， 2 以進行語音輸入, 結束 以結束對話：1
請輸入您的問題：你好啊
嗨，有什麼我可以幫助你的？
輸入 1 以進行文字輸入， 2 以進行語音輸入, 結束 以結束對話：1
請輸入您的問題：我有一些問題
這是一個很常見的感受，很多人在面對憂鬱時都感到困擾。你考慮過尋求專業心理醫生或治療師的幫助嗎？他們通常能夠提供支持並理解你的感受。
輸入 1 以進行文字輸入， 2 以進行語音輸入, 結束 以結束對話：1
請輸入您的問題：我可以怎麼做？
尋求幫助是一個非常勇敢的決定，我為你這樣的勇氣感到驕傲。其實，研究表明，透過尋求支持與治療，情緒會有很大改善​。你不需要獨自承受這些困難，我們可以一起尋找適合的解決方案。
輸入 1 以進行文字輸入， 2 以進行語音輸入, 結束 以結束對話：1
請輸入您的問題：謝謝您
我的榮幸！
輸入 1 以進行文字輸入， 2 以進行語音輸入, 結束 以結束對話：結束
再見！
